In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
os.chdir('data')

In [3]:
[file for file in os.listdir() if(file.endswith('.csv'))]

['Median Incomes.csv',
 'Median Monthly Rent.csv',
 'Home Ownership.csv',
 'Total Population.csv',
 'Uninsured.csv',
 'Unemployment Rate.csv',
 'Severe Rent Burden.csv',
 'Educational Attainment.csv',
 'Overcrowded Rental Housing.csv',
 'SNAP (Food Stamps).csv',
 'Household Income.csv',
 'Public Assistance.csv',
 'Employment Population Ratio.csv',
 'Total Population by Race_Ethnicity.csv']

In [4]:
dfs = {}
for file in os.listdir():
    if (file.endswith('.csv')):
        key = file[:-4].replace(' ', '_').lower()
        #print(key)
        dfs[key] = pd.read_csv(file, skiprows=4, encoding='utf8', low_memory=False)
        #print(f"{key} - {sorted(set(dfs[key]['TimeFrame'].tolist()))}")

In [5]:
dfs.keys()

dict_keys(['median_incomes', 'median_monthly_rent', 'home_ownership', 'total_population', 'uninsured', 'unemployment_rate', 'severe_rent_burden', 'educational_attainment', 'overcrowded_rental_housing', 'snap_(food_stamps)', 'household_income', 'public_assistance', 'employment_population_ratio', 'total_population_by_race_ethnicity'])

In [6]:
for key in dfs.keys():
    print(f'{key} - {dfs[key].columns.tolist()}')

median_incomes - ['Location', 'Household Type', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
median_monthly_rent - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
home_ownership - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
total_population - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
uninsured - ['Location', 'Age Group', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
unemployment_rate - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
severe_rent_burden - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
educational_attainment - ['Location', 'Education Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
overcrowded_rental_housing - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
snap_(food_stamps) - ['Location', 'Recipient', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
household_income - ['Location', 'Income Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
public_assistance - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
employm

In [7]:
for key in dfs.keys():
    if('Number' in list(set(dfs[key]['DataFormat'].tolist())) ):
        print(key)
        

total_population
uninsured
educational_attainment
snap_(food_stamps)
household_income
public_assistance
total_population_by_race_ethnicity


In [8]:
standard = {}
extra = {}
for key in dfs.keys():
    if(dfs[key].columns[1] == 'TimeFrame' and key != 'public_assistance'):
        print(f'{key} - {dfs[key].columns.tolist()}')
        standard[key] = dfs[key]
        if(key == 'total_population'):
            standard[key] = dfs[key][~dfs[key].Fips.str.contains('uhf')]
            standard[key].Fips = standard[key].Fips.astype(int)
        if(key == 'median_monthly_rent'):
            standard[key]['DataFormat'] = 'Number'
    else:
        extra[key] = dfs[key]
    #print(dfs[key].head(2))

median_monthly_rent - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
home_ownership - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
total_population - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
unemployment_rate - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
severe_rent_burden - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
overcrowded_rental_housing - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
employment_population_ratio - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']


/Users/tavares/opt/anaconda3/envs/PythonML/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
print('standard: ', list(standard.keys()))
print('')
print('extra: ', list(extra.keys()))

standard:  ['median_monthly_rent', 'home_ownership', 'total_population', 'unemployment_rate', 'severe_rent_burden', 'overcrowded_rental_housing', 'employment_population_ratio']

extra:  ['median_incomes', 'uninsured', 'educational_attainment', 'snap_(food_stamps)', 'household_income', 'public_assistance', 'total_population_by_race_ethnicity']


In [10]:
df = standard['median_monthly_rent'][['TimeFrame','Fips','Location']]

In [11]:
df = df.merge(standard['total_population'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Population'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

In [12]:
df = df.merge(standard['median_monthly_rent'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Rent'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

In [13]:
df = df.merge(standard['severe_rent_burden'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Rent_Burden'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)
df = df.merge(standard['overcrowded_rental_housing'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Overcrowded'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

In [14]:
df = df.merge(standard['home_ownership'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Owner'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

In [15]:
df = df.merge(standard['employment_population_ratio'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Employment_Ratio'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)
df = df.merge(standard['unemployment_rate'][['TimeFrame','Fips','Data']].rename(columns={'Data':'Unemployment'}),on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

In [16]:
print(df.shape)
df.head()

(905, 10)


,TimeFrame,Fips,Location,Population,Rent,Rent_Burden,Overcrowded,Owner,Employment_Ratio,Unemployment
0,2005,101,Battery Park/Tribeca,51150.45963,2191.21645,0.17644,0.03794,0.26053,0.751,0.033
1,2005,102,Greenwich Village,75508.54037,2191.21645,0.17644,0.03794,0.26053,0.751,0.033
2,2005,103,Lower East Side,145556.00000,817.19484,0.23809,0.08153,0.13708,0.564,0.088
3,2005,104,Chelsea/Clinton,88754.39184,1781.33008,0.19318,0.04339,0.20554,0.677,0.066
4,2005,105,Midtown Business District,44420.60816,1781.33008,0.19318,0.04339,0.20554,0.677,0.066


In [17]:
for key in extra:
    print(f'{key} - {extra[key].columns.tolist()}')

median_incomes - ['Location', 'Household Type', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
uninsured - ['Location', 'Age Group', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
educational_attainment - ['Location', 'Education Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
snap_(food_stamps) - ['Location', 'Recipient', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
household_income - ['Location', 'Income Level', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
public_assistance - ['Location', 'TimeFrame', 'DataFormat', 'Data', 'Fips']
total_population_by_race_ethnicity - ['Location', 'Race/Ethnicity', 'TimeFrame', 'DataFormat', 'Data', 'Fips']


In [18]:
median_incomes = extra['median_incomes'][extra['median_incomes']['Household Type']=='All Households']
median_incomes = median_incomes[['TimeFrame','Fips','DataFormat','Data']].set_index(['TimeFrame','Fips','DataFormat']).unstack()
median_incomes.columns = ['Median_Income']
#median_incomes['Type'] = 'median_income'
median_incomes = median_incomes.reset_index()
print(median_incomes.shape)
median_incomes.head()
df = df.merge(median_incomes,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(910, 3)


In [19]:
uninsured = extra['uninsured'][extra['uninsured']['Age Group']=='All Ages']
uninsured = uninsured[['TimeFrame','Fips','DataFormat','Data']].set_index(['TimeFrame','Fips','DataFormat']).unstack()
uninsured.columns = ['Uninsured_No','Uninsured_Pct']
#uninsured['Type'] = 'uninsured'
uninsured = uninsured.reset_index()
print(uninsured.shape)
uninsured.head()
df = df.merge(uninsured,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(715, 4)


In [20]:
education = extra['educational_attainment']
education = education[['TimeFrame','Fips','Education Level','DataFormat','Data']].set_index(['TimeFrame','Fips','Education Level','DataFormat']).unstack()
education.columns = ['Education_No','Education_Pct']
#education['Type'] = 'education'
education = education.reset_index().rename(columns={'Education Level': 'Education_Level'})
print(education.shape)
education.head()
df = df.merge(education,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(4225, 5)


In [21]:
snap = extra['snap_(food_stamps)'][extra['snap_(food_stamps)']['Recipient']=='Households']
snap = snap[['TimeFrame','Fips','DataFormat','Data']].set_index(['TimeFrame','Fips','DataFormat']).unstack()
snap.columns = ['Snap_No','Snap_Pct']
snap['Type'] = 'snap'
snap = snap.reset_index()
print(snap.shape)
snap.head()
df = df.merge(snap,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(715, 5)


In [22]:
households = extra['household_income']
households = households[['TimeFrame','Fips','Income Level','DataFormat','Data']].set_index(['TimeFrame','Fips','Income Level','DataFormat']).unstack()
households.columns = ['Income_No','Income_Pct']
#households['Type'] = 'household_income'
households = households.reset_index().rename(columns={'Income Level': 'Income_Level'})
print(households.shape)
households.head()
df = df.merge(households,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(7280, 5)


In [23]:
public_assistance = extra['public_assistance']
public_assistance = public_assistance[['TimeFrame','Fips','DataFormat','Data']].set_index(['TimeFrame','Fips','DataFormat']).unstack()
public_assistance.columns = ['PA_No','PA_Pct']
#public_assistance['Type'] = 'public_assistance'
public_assistance = public_assistance.reset_index()
print(public_assistance.shape)
public_assistance.head()
df = df.merge(public_assistance,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(455, 4)


In [24]:
pop_by_race = extra['total_population_by_race_ethnicity']
pop_by_race = pop_by_race[['TimeFrame','Fips','Race/Ethnicity','DataFormat','Data']].set_index(['TimeFrame','Fips','Race/Ethnicity','DataFormat']).unstack()
pop_by_race.columns = ['Race_No','Race_Pct']
#pop_by_race['Type'] = 'population_by_race'
pop_by_race = pop_by_race.reset_index().rename(columns={'Race/Ethnicity': 'Race'})
print(pop_by_race.shape)
pop_by_race.head()
df = df.merge(pop_by_race,on=['TimeFrame','Fips'],how='inner', left_index=False, right_index=False)

(3250, 5)


In [25]:
sorted(pop_by_race['TimeFrame'].value_counts().index)

[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [26]:
print(df.shape)
print(df.columns.tolist())
df.head(10)

(51800, 27)
['TimeFrame', 'Fips', 'Location', 'Population', 'Rent', 'Rent_Burden', 'Overcrowded', 'Owner', 'Employment_Ratio', 'Unemployment', 'Median_Income', 'Uninsured_No', 'Uninsured_Pct', 'Education_Level', 'Education_No', 'Education_Pct', 'Snap_No', 'Snap_Pct', 'Type', 'Income_Level', 'Income_No', 'Income_Pct', 'PA_No', 'PA_Pct', 'Race', 'Race_No', 'Race_Pct']


,TimeFrame,Fips,Location,Population,Rent,Rent_Burden,Overcrowded,Owner,Employment_Ratio,Unemployment,...,Snap_Pct,Type,Income_Level,Income_No,Income_Pct,PA_No,PA_Pct,Race,Race_No,Race_Pct
0,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$100,000 to $199,999",7638.30279,0.25258,474.0,0.00838,Asian,8525.95160,0.14727
1,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$100,000 to $199,999",7638.30279,0.25258,474.0,0.00838,Black,1287.05039,0.02223
2,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$100,000 to $199,999",7638.30279,0.25258,474.0,0.00838,Latino,3291.73131,0.05686
3,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$100,000 to $199,999",7638.30279,0.25258,474.0,0.00838,Other,1439.29952,0.02486
4,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$100,000 to $199,999",7638.30279,0.25258,474.0,0.00838,White,43348.60008,0.74878
5,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$15,000 to $24,999",1777.72068,0.05878,474.0,0.00838,Asian,8525.95160,0.14727
6,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$15,000 to $24,999",1777.72068,0.05878,474.0,0.00838,Black,1287.05039,0.02223
7,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$15,000 to $24,999",1777.72068,0.05878,474.0,0.00838,Latino,3291.73131,0.05686
8,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$15,000 to $24,999",1777.72068,0.05878,474.0,0.00838,Other,1439.29952,0.02486
9,2009,101,Battery Park/Tribeca,56591.85182,2305.38388,0.21577,0.02959,0.28839,0.707,0.07583,...,0.05719,snap,"$15,000 to $24,999",1777.72068,0.05878,474.0,0.00838,White,43348.60008,0.74878


In [27]:
"""
median_incomes - 2005-2018
monthly_rent - 2009-2018
arrests - 2000-2019
child_population_by_race_ethnicity - 2005-2018
median_monthly_rent - 2005-2018
home_ownership - 2005-2018
total_population - 2005-2018
child_population - 2005-2018
uninsured - 2008-2018
public_assistance_by_program_type - 2005-2017
unemployment_rate_ - 2005-2018
severe_rent_burden - 2005- 2018
educational_attainment - 2006-2018
overcrowded_rental_housing - 2005-2018
snap_(food_stamps) - 2001-2019
household_income - 2005-2018
public_assistance - 2005-2017
employment_population_ratio - 2005-2018
total_population_by_race_ethnicity - 2009-2018
"""

'\nmedian_incomes - 2005-2018\nmonthly_rent - 2009-2018\narrests - 2000-2019\nchild_population_by_race_ethnicity - 2005-2018\nmedian_monthly_rent - 2005-2018\nhome_ownership - 2005-2018\ntotal_population - 2005-2018\nchild_population - 2005-2018\nuninsured - 2008-2018\npublic_assistance_by_program_type - 2005-2017\nunemployment_rate_ - 2005-2018\nsevere_rent_burden - 2005- 2018\neducational_attainment - 2006-2018\novercrowded_rental_housing - 2005-2018\nsnap_(food_stamps) - 2001-2019\nhousehold_income - 2005-2018\npublic_assistance - 2005-2017\nemployment_population_ratio - 2005-2018\ntotal_population_by_race_ethnicity - 2009-2018\n'

In [28]:
#df.to_csv('../data.csv')